# Jointure MapReduce

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Using cached version of Emiasd-Flight-Data-Analysis.jar


error: error while loading FlightDataLoader, class file '/tmp/toree-tmp-dir14488093077525536507/toree_add_jars/Emiasd-Flight-Data-Analysis.jar(com/flightdelay/data/loaders/FlightDataLoader.class)' has location not matching its contents: contains class FlightDataLoader


Using cached version of Emiasd-Flight-Data-Analysis.jar


In [48]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader
import org.apache.spark.sql.functions._
import java.sql.Date

//Env Configuration
val args: Array[String] = Array("juniper")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

args = Array(juniper)
spark = org.apache.spark.sql.SparkSession@708cf8ec
session = org.apache.spark.sql.SparkSession@708cf8ec
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan...

# Chargement Flight et Weather

In [59]:
// Chargement de Flight Data
val processedFlightPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightDF = spark.read.parquet(processedFlightPath)

val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDF = spark.read.parquet(processedWeatherPath)



processedFlightPath = /home/jovyan/work/output/common/data/processed_flights.parquet
flightDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]
processedWeatherPath = /home/jovyan/work/output/common/data/processed_weather.parquet
weatherDF = [WBAN: string, Date: date ... 65 more fields]


[WBAN: string, Date: date ... 65 more fields]

In [60]:
weatherDF.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string (

In [61]:
flightDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- ORIGIN_TIMEZONE: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- DEST_TIMEZONE: integer (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- UTC_ARR_TIME: string (nullable = true)
 |-- UTC_ARR_DATE: date (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_ARR_DATE: date (nullable = true)
 |-- feature_arrival_hour: integer (nullable = true)
 |-- feature_utc_arrival_hour: integer (

## Weather

In [1]:
import java.sql.Date

val weatherColumns = Seq(
  "WBAN",
  "Date",
  "Time",
  "StationType",
  "SkyCondition",
  "Visibility",
)

val weatherData = weatherDF
  .select(weatherColumns.head, weatherColumns.tail: _*)
  .filter(
    col("Date") === Date.valueOf("2012-01-09") &&
    (col("WBAN") === "13869" || col("WBAN") === "13874")
  )
  .orderBy("WBAN", "Date", "Time")
weatherData.show(999)

Unknown Error: <console>:34: error: not found: value weatherDF
       val weatherData = weatherDF
                         ^
<console>:37: error: not found: value col
           col("Date") === Date.valueOf("2012-01-09") &&
           ^
<console>:38: error: not found: value col
           (col("WBAN") === "13869" || col("WBAN") === "13874")
            ^
<console>:38: error: not found: value col
           (col("WBAN") === "13869" || col("WBAN") === "13874")
                                       ^


## Flights

In [63]:
val flightColumns = Seq(
  "FL_DATE",
  "OP_CARRIER_AIRLINE_ID",
  "OP_CARRIER_FL_NUM",
  "ORIGIN_AIRPORT_ID",
  "DEST_AIRPORT_ID",
  "CRS_DEP_TIME",
  "ARR_DELAY_NEW",
  "CRS_ELAPSED_TIME",
  "ORIGIN_WBAN",
  "ORIGIN_TIMEZONE",
  "DEST_WBAN",
  "DEST_TIMEZONE",
  "UTC_CRS_DEP_TIME",
  "UTC_FL_DATE",
  "UTC_ARR_TIME",
  "UTC_ARR_DATE",
  "CRS_ARR_TIME",
  "CRS_ARR_DATE",  
  
  "feature_departure_hour_rounded",
  "feature_utc_departure_hour_rounded",
  "feature_arrival_hour_rounded",
  "feature_utc_arrival_hour_rounded",
    
  "label_is_delayed_15min"
)

val flightData = flightDF
  .select(flightColumns.head, flightColumns.tail: _*)
  .filter(
    col("UTC_FL_DATE") === "2012-01-09" &&
    col("OP_CARRIER_AIRLINE_ID") === 20366 &&
    col("OP_CARRIER_FL_NUM") === 5334 && 
    col("ORIGIN_WBAN") === 13869  
  )

flightData.show(10, 100, true) 


-RECORD 0----------------------------------------
 FL_DATE                            | 2012-01-09 
 OP_CARRIER_AIRLINE_ID              | 20366      
 OP_CARRIER_FL_NUM                  | 5334       
 ORIGIN_AIRPORT_ID                  | 10146      
 DEST_AIRPORT_ID                    | 10397      
 CRS_DEP_TIME                       | 1155       
 ARR_DELAY_NEW                      | 115.0      
 CRS_ELAPSED_TIME                   | 57.0       
 ORIGIN_WBAN                        | 13869      
 ORIGIN_TIMEZONE                    | -5         
 DEST_WBAN                          | 13874      
 DEST_TIMEZONE                      | -5         
 UTC_CRS_DEP_TIME                   | 1655       
 UTC_FL_DATE                        | 2012-01-09 
 UTC_ARR_TIME                       | 1825       
 UTC_ARR_DATE                       | 2012-01-09 
 CRS_ARR_TIME                       | 1340       
 CRS_ARR_DATE                       | 2012-01-09 
 feature_departure_hour_rounded     | 1200       


flightColumns = List(FL_DATE, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID, CRS_DEP_TIME, ARR_DELAY_NEW, CRS_ELAPSED_TIME, ORIGIN_WBAN, ORIGIN_TIMEZONE, DEST_WBAN, DEST_TIMEZONE, UTC_CRS_DEP_TIME, UTC_FL_DATE, UTC_ARR_TIME, UTC_ARR_DATE, CRS_ARR_TIME, CRS_ARR_DATE, feature_departure_hour_rounded, feature_utc_departure_hour_rounded, feature_arrival_hour_rounded, feature_utc_arrival_hour_rounded, label_is_delayed_15min)
flightData = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 21 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 21 more fields]

# Jointure Map Reduce 1

In [20]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import scala.collection.JavaConverters._

// Recommandé: travailler en UTC pour éviter les surprises
spark.conf.set("spark.sql.session.timeZone", "UTC")
spark.conf.set("spark.sql.crossJoin.enabled", "true")
spark.conf.set("spark.sql.debug.maxToStringFields", "100000")


In [21]:
// -----------------------------------------
// 0) Normalisation du dataset Weather
//    - DATE  -> date
//    - TIME  -> heure 2 chiffres "HH"
// -----------------------------------------
val weatherNorm = weatherData
  // DATE peut être string => on force en date (UTC)
  .withColumn("W_DATE", to_date(col("DATE")))
  // TIME peut être "0", "00", "0030", 0, etc.
  .withColumn("TIME_STR", col("TIME").cast("string"))
  .withColumn(
    "W_HOUR",
    when(length(col("TIME_STR")) >= 2, substring(col("TIME_STR"), 1, 2))
      .otherwise(lpad(col("TIME_STR"), 2, "0"))
  )
  .select(
    col("WBAN"),
    col("W_DATE"),
    col("W_HOUR"),
    // garde ce qui t'intéresse côté météo :
    col("*") // tu peux remplacer par les colonnes utiles: vent, visib, etc.
  )
weatherNorm.show(12)

weatherNorm = [WBAN: string, W_DATE: date ... 10 more fields]


+-----+----------+------+-----+----------+----+-----------+-------------+----------+----------+--------+------+
| WBAN|    W_DATE|W_HOUR| WBAN|      Date|Time|StationType| SkyCondition|Visibility|    W_DATE|TIME_STR|W_HOUR|
+-----+----------+------+-----+----------+----+-----------+-------------+----------+----------+--------+------+
|13869|2012-01-09|    00|13869|2012-01-09|0000|         12|          CLR|      10.0|2012-01-09|    0000|    00|
|13869|2012-01-09|    01|13869|2012-01-09|0100|         12|BKN050 OVC100|      10.0|2012-01-09|    0100|    01|
|13869|2012-01-09|    02|13869|2012-01-09|0200|         12|       OVC110|      10.0|2012-01-09|    0200|    02|
|13869|2012-01-09|    03|13869|2012-01-09|0300|         12|       SCT001|       7.0|2012-01-09|    0300|    03|
|13869|2012-01-09|    04|13869|2012-01-09|0400|         12|BKN001 OVC005|       1.5|2012-01-09|    0400|    04|
|13869|2012-01-09|    05|13869|2012-01-09|0500|         12|       OVC100|      1.75|2012-01-09|    0500|

[WBAN: string, W_DATE: date ... 10 more fields]

In [22]:
// Option: pour le shuffle, tu peux aider Spark à partitionner:
val weatherPart = weatherNorm.repartition(col("WBAN"), col("W_DATE"))

weatherPart = [WBAN: string, W_DATE: date ... 10 more fields]


[WBAN: string, W_DATE: date ... 10 more fields]

In [30]:
// ───────────────────────────────────────────────────────────────────────────────
// 0) Crée un identifiant de vol déterministe "flight_id"
//    (tu peux remplacer par monotonically_increasing_id() si tu préfères)
// ───────────────────────────────────────────────────────────────────────────────
val flights0 = flightData
  .withColumn(
    "flight_id",
    concat_ws("_",
      date_format(col("UTC_FL_DATE").cast("date"), "yyyyMMdd"),
      col("OP_CARRIER_AIRLINE_ID").cast("string"),
      col("OP_CARRIER_FL_NUM").cast("string"),
      col("ORIGIN_WBAN").cast("string"),
      col("DEST_WBAN").cast("string"),
      date_format(col("UTC_CRS_DEP_TIME"), "HHmm")
    )
  )

flights0.show(10, 100, true)

-RECORD 0------------------------------------------------------------------
 UTC_FL_DATE                        | 2012-01-09                           
 UTC_CRS_DEP_TIME                   | 1655                                 
 FL_DATE                            | 2012-01-09                           
 CRS_DEP_TIME                       | 1155                                 
 ORIGIN_WBAN                        | 13869                                
 DEST_WBAN                          | 13874                                
 CRS_ELAPSED_TIME                   | 57.0                                 
 feature_utc_departure_hour         | 16                                   
 OP_CARRIER_AIRLINE_ID              | 20366                                
 OP_CARRIER_FL_NUM                  | 5334                                 
 feature_flight_day_of_week         | 2                                    
 feature_is_weekend                 | 0                                    
 feature_dep

flights0 = [UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 14 more fields]


[UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 14 more fields]

In [31]:

// ───────────────────────────────────────────────────────────────────────────────
// 1) Base temporelle départ (UTC) : "yyyy-MM-dd HH" → epoch (secondes)
// ───────────────────────────────────────────────────────────────────────────────
val flightsBase = flights0
  .withColumn(
    "FL_DATE_STR",
    when(col("UTC_FL_DATE").cast("date").isNotNull,
         date_format(col("UTC_FL_DATE").cast("date"), "yyyy-MM-dd"))
    .otherwise(col("UTC_FL_DATE").cast("string"))
  )
  .withColumn("DEP_HOUR_2D", lpad(col("feature_utc_departure_hour_rounded").cast("string"), 2, "0"))
  .withColumn("base_epoch",
    unix_timestamp(concat_ws(" ", col("FL_DATE_STR"), col("DEP_HOUR_2D")), "yyyy-MM-dd HH")
  )
  // IMPORTANT : on ne garde QUE ce qu'on va utiliser ensuite (dont flight_id !)
  .select(
    col("flight_id"),
    col("ORIGIN_WBAN"),
    col("DEST_WBAN"),
    col("FL_DATE_STR"),
    col("DEP_HOUR_2D"),
    col("base_epoch"),
    col("label_is_delayed_15min")  
  )

flightsBase.show(10, 100, true)

flightsBase = [flight_id: string, ORIGIN_WBAN: string ... 5 more fields]


-RECORD 0------------------------------------------------------
 flight_id              | 20120109_20366_5334_13869_13874_0000 
 ORIGIN_WBAN            | 13869                                
 DEST_WBAN              | 13874                                
 FL_DATE_STR            | 2012-01-09                           
 DEP_HOUR_2D            | 17                                   
 base_epoch             | 1326128400                           
 label_is_delayed_15min | 1                                    



[flight_id: string, ORIGIN_WBAN: string ... 5 more fields]

In [25]:
// ───────────────────────────────────────────────────────────────────────────────
// 2) Générer les 12 heures cibles t-1h..t-12h (crossJoin offsets 1..12)
// ───────────────────────────────────────────────────────────────────────────────
val offsets = flightsBase.sparkSession.range(1, 13).toDF("h")  // 1..12

val originTargets = flightsBase
  .crossJoin(offsets)
  .withColumn("target_epoch", col("base_epoch") - col("h") * lit(3600))
  .withColumn("TARGET_DATE", to_date(from_unixtime(col("target_epoch"))))
  .withColumn("TARGET_HOUR_2D", date_format(from_unixtime(col("target_epoch")), "HH"))
  .select(
    col("flight_id"),                               
    col("ORIGIN_WBAN").as("WBAN_KEY"),
    col("h").cast("int").as("o_h"),
    col("TARGET_DATE"),
    col("TARGET_HOUR_2D"),
    col("label_is_delayed_15min")  
  )

// (debug)
originTargets.show(12, truncate = false)

offsets = [h: bigint]
originTargets = [flight_id: string, WBAN_KEY: string ... 4 more fields]


+------------------------------------+--------+---+-----------+--------------+----------------------+
|flight_id                           |WBAN_KEY|o_h|TARGET_DATE|TARGET_HOUR_2D|label_is_delayed_15min|
+------------------------------------+--------+---+-----------+--------------+----------------------+
|20120109_20366_5334_13869_13874_0000|13869   |1  |2012-01-09 |16            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |2  |2012-01-09 |15            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |3  |2012-01-09 |14            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |4  |2012-01-09 |13            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |5  |2012-01-09 |12            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |6  |2012-01-09 |11            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |7  |2012-01-09 |10            |1   

[flight_id: string, WBAN_KEY: string ... 4 more fields]

In [26]:
// ───────────────────────────────────────────────────────────────────────────────
// 3) Normalisation météo (si pas déjà fait) et jointure exacte (WBAN, DATE, HOUR)
// ───────────────────────────────────────────────────────────────────────────────

val originJoined = originTargets
  .join(
    weatherNorm,
    originTargets.col("WBAN_KEY") === weatherNorm.col("WBAN") &&
    originTargets.col("TARGET_DATE") === weatherNorm.col("W_DATE") &&
    originTargets.col("TARGET_HOUR_2D") === weatherNorm.col("W_HOUR"),
    "inner"
  )
  .select(
    col("flight_id"),
    col("o_h"),
    col("WBAN_KEY").as("WBAN"),
    col("TARGET_DATE").as("o_target_date"),
    col("TARGET_HOUR_2D").as("o_target_hour"),
    col("W_DATE"), col("W_HOUR"),
    col("DATE"), col("TIME"),
    col("StationType"),
    col("SkyCondition"),
    col("Visibility"),
    //col("feature_ceiling"),  
  ).orderBy(col("flight_id"), col("o_target_hour"))

originJoined.show(20, false)

originJoined = [flight_id: string, o_h: int ... 10 more fields]


+------------------------------------+---+-----+-------------+-------------+----------+------+----------+----+-----------+------------+----------+
|flight_id                           |o_h|WBAN |o_target_date|o_target_hour|W_DATE    |W_HOUR|DATE      |TIME|StationType|SkyCondition|Visibility|
+------------------------------------+---+-----+-------------+-------------+----------+------+----------+----+-----------+------------+----------+
|20120109_20366_5334_13869_13874_0000|12 |13869|2012-01-09   |05           |2012-01-09|05    |2012-01-09|0500|12         |OVC100      |1.75      |
|20120109_20366_5334_13869_13874_0000|11 |13869|2012-01-09   |06           |2012-01-09|06    |2012-01-09|0600|12         |OVC110      |9.0       |
|20120109_20366_5334_13869_13874_0000|10 |13869|2012-01-09   |07           |2012-01-09|07    |2012-01-09|0700|12         |OVC110      |10.0      |
|20120109_20366_5334_13869_13874_0000|9  |13869|2012-01-09   |08           |2012-01-09|08    |2012-01-09|0800|12      

[flight_id: string, o_h: int ... 10 more fields]

In [29]:
// ───────────────────────────────────────────────────────────────────────────────
// 4) (Option) Regrouper en liste par vol (gardez o_h pour trier côté lecteur)
// ───────────────────────────────────────────────────────────────────────────────
val originAgg = originJoined
  .withColumn("o_pack", struct(
    col("o_h"),
    col("o_target_date"),
    col("o_target_hour"),
    col("W_DATE"),
    col("W_HOUR"),
    col("SkyCondition"),
    col("Visibility"),
    // + tes variables météo utiles
  ))
  .groupBy("flight_id")                       
  .agg(collect_list(col("o_pack")).as("W_o_list"))

originAgg.show(10, 10000, true)

originAgg = [flight_id: string, W_o_list: array<struct<o_h:int,o_target_date:date,o_target_hour:string,W_DATE:date,W_HOUR:string,SkyCondition:string,Visibility:double>>]


-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 flight_id | 20120109_20366_5334_13869_13874_0000                                                                                                                                                                                                                                                                                                                                                   

[flight_id: string, W_o_list: array<struct<o_h:int,o_target_date:date,o_target_hour:string,W_DATE:date,W_HOUR:string,SkyCondition:string,Visibility:double>>]

# Jointure Map Reduce 2

In [67]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

object FlightWeatherJoinner {

  /**
   * Jointure spatio-temporelle entre vols et observations météo pour ORIGINE et DESTINATION
   * 
   * @param flightDF DataFrame des vols (avec feature_*_hour_rounded déjà calculées)
   * @param weatherDF DataFrame météo
   * @return DataFrame avec origin_weather_observations et dest_weather_observations
   */
  def joinFlightsWithWeather(
    flightDF: DataFrame, 
    weatherDF: DataFrame
  ): DataFrame = {
    
    // Première jointure : aéroport d'origine
    val withOriginWeather = joinWeatherForAirport(flightDF, weatherDF, "origin")
    
    // Deuxième jointure : aéroport de destination
    val withBothWeather = joinWeatherForAirport(withOriginWeather, weatherDF, "destination")
    
    withBothWeather
  }

  /**
   * Jointure spatio-temporelle pour UN aéroport (origine OU destination)
   * 
   * @param flightDF DataFrame des vols
   * @param weatherDF DataFrame météo
   * @param airportType "origin" ou "destination"
   * @return DataFrame avec colonne weather_observations nommée selon le type
   */
  def joinWeatherForAirport(
    flightDF: DataFrame,
    weatherDF: DataFrame,
    airportType: String
  ): DataFrame = {
    
    require(
      airportType == "origin" || airportType == "destination",
      "airportType doit être 'origin' ou 'destination'"
    )
    
    import flightDF.sparkSession.implicits._
    
    val isOrigin = airportType == "origin"
    
    // Déterminer les colonnes à utiliser selon le type d'aéroport
    val (wbanCol, dateCol, timeRoundedCol) = if (isOrigin) {
      ("ORIGIN_WBAN", "UTC_FL_DATE", "feature_utc_departure_hour_rounded")
    } else {
      ("DEST_WBAN", "UTC_ARR_DATE", "feature_utc_arrival_hour_rounded")
    }
    
    // ============================================
    // PHASE MAP - Préparation des données
    // ============================================
    
    // MAP Weather (OT): Tagger les observations météo
    val weatherTagged = weatherDF
      .withColumn("table_tag", lit("OT"))
      .withColumn("join_key_wban", col("WBAN"))
      .withColumn("join_key_date", col("Date"))
    
    // MAP Flights (FT): Tagger les vols - utiliser l'heure arrondie précalculée
    val flightTagged = flightDF
      .filter(col(wbanCol).isNotNull && col(timeRoundedCol).isNotNull)
      .withColumn("table_tag", lit("FT"))
      .withColumn("join_key_wban", col(wbanCol))
      .withColumn("join_key_date", col(dateCol))
      .withColumn("time_rounded", col(timeRoundedCol))  // ← Utiliser la colonne précalculée
        
    // Dupliquer UNIQUEMENT les vols de début de journée.
    // Vols de 00h à 11h
    val flightsNeedingDuplication = flightTagged
      .filter(col("time_rounded") / 100 < 12)  // Vols de 00h à 11h
      .withColumn("join_key_date", date_sub(col("join_key_date"), 1))  // Date J-1
    
    // Union : vols normaux + vols dupliqués pour le lendemain
    val flightComplete = flightTagged.union(flightsNeedingDuplication)
    
    // ============================================
    // PHASE SHUFFLE - Préparation pour jointure
    // ============================================
    
    val weatherForJoin = weatherTagged
      .select(
        col("join_key_wban").as("key_wban"),
        col("join_key_date").as("key_date"),
        col("table_tag"),
        col("WBAN"),
        col("Date"),
        col("Time").cast("int").as("Time"),
        col("StationType"),
        col("SkyCondition"),
        col("Visibility")
      )
    
    // Récupérer toutes les colonnes du vol original
    val allFlightCols = flightComplete.columns.toSeq
    val excludeCols = Set("table_tag", "join_key_wban", "join_key_date", "time_rounded")
    val flightDataCols = allFlightCols.filterNot(excludeCols.contains)
    
    val flightForJoin = flightComplete
      .select(
        Seq(
          col("join_key_wban").as("key_wban"),
          col("join_key_date").as("key_date"),
          col("time_rounded")
        ) ++ flightDataCols.map(c => col(c)): _*
      )
    
    // ============================================
    // PHASE REDUCE - Jointure et Agrégation
    // ============================================
    
    // Grouper toutes les observations météo par (WBAN, Date)
    val weatherGrouped = weatherForJoin
      .groupBy("key_wban", "key_date")
      .agg(
        collect_list(
          struct(
            col("WBAN"),
            col("Date"),
            col("Time"),
            col("StationType"),
            col("SkyCondition"),
            col("Visibility")
          )
        ).as("all_weather_obs")
      )
    
    // Jointure vols + météo groupée
    val joined = flightForJoin
      .join(
        weatherGrouped,
        flightForJoin("key_wban") === weatherGrouped("key_wban") &&
        flightForJoin("key_date") === weatherGrouped("key_date"),
        "left"
      )
    
    // Nom de la colonne de sortie selon le type d'aéroport
    val weatherColName = s"${airportType}_weather_observations"
    
    // Filtrer pour garder seulement les 13 heures nécessaires
    val joinedWithFiltered = joined
      .withColumn("required_hours",
        expr("""
          transform(
            sequence(0, 12),
            i -> cast((time_rounded / 100 - (12 - i) + 24) % 24 * 100 as int)
          )
        """)
      )
      .withColumn(weatherColName,
        expr("""
          filter(
            all_weather_obs,
            obs -> array_contains(required_hours, obs.Time)
          )
        """)
      )
    
    // Sélectionner les colonnes finales
    val finalColsSeq = (flightDataCols :+ weatherColName).map(c => col(c))
    val result = joinedWithFiltered
      .select(finalColsSeq: _*)
      .filter(size(col(weatherColName)) > 0)
    
    result
  }
}


// Première jointure : aéroport d'origine
val joinedData = FlightWeatherJoinner.joinFlightsWithWeather(flightData, weatherData)

joinedData.show(20, 10000, true)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 FL_DATE                            | 2012-01-09                                                                                                                                                                  

defined object FlightWeatherJoinner
joinedData = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 23 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 23 more fields]

# Jointure Map Reduce 3

In [84]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

object FlightWeatherJoinner {

  /**
   * Jointure spatio-temporelle entre vols et observations météo pour ORIGINE et DESTINATION
   * 
   * @param flightDF DataFrame des vols (avec feature_*_hour_rounded déjà calculées)
   * @param weatherDF DataFrame météo
   * @param weatherDepthHours Nombre d'heures d'observations météo à récupérer avant le vol (par défaut: 12)
   * @param removeLeakageColumns Si true, supprime automatiquement les colonnes qui causent du data leakage (par défaut: false)
   * @param selectedFlightColumns Colonnes du DataFrame vols à conserver (si None, toutes les colonnes)
   * @param selectedWeatherColumns Colonnes du DataFrame météo à inclure dans les observations (si None, toutes les colonnes)
   * @return DataFrame avec origin_weather_observations et dest_weather_observations
   */
  def joinFlightsWithWeather(
    flightDF: DataFrame, 
    weatherDF: DataFrame,
    weatherDepthHours: Int = 12,
    removeLeakageColumns: Boolean = true,
    selectedFlightColumns: Option[Seq[String]] = None,
    selectedWeatherColumns: Option[Seq[String]] = None
  ): DataFrame = {
    
    require(weatherDepthHours > 0 && weatherDepthHours <= 24, 
      s"weatherDepthHours doit être entre 1 et 24, valeur fournie: $weatherDepthHours")
    
    // Première jointure : aéroport d'origine
    val withOriginWeather = joinWeatherForAirport(
      flightDF, 
      weatherDF, 
      "origin",
      weatherDepthHours,
      selectedFlightColumns,
      selectedWeatherColumns
    )
    
    // Deuxième jointure : aéroport de destination
    val withBothWeather = joinWeatherForAirport(
      withOriginWeather, 
      weatherDF, 
      "destination",
      weatherDepthHours,
      selectedFlightColumns,
      selectedWeatherColumns
    )
    
      // Supprimer les colonnes de leakage si demandé
      if (removeLeakageColumns) {
        removeDataLeakageColumns(
          withBothWeather, 
          keepRelativeTime = removeLeakageColumns,  // ou true si tu veux garder hours_before_flight
          selectedWeatherColumns = selectedWeatherColumns
        )
      } else {
        withBothWeather
      }
  }

    /**
     * Supprime les colonnes qui peuvent causer du data leakage
     * 
     * @param df DataFrame avec potentiellement des colonnes de leakage
     * @param keepRelativeTime Si true, transforme Time en hours_before_flight. Si false, supprime Time/Date complètement
     * @param selectedWeatherColumns Les colonnes météo qui ont été sélectionnées lors de la jointure
     * @return DataFrame nettoyé sans colonnes de leakage
     */
    def removeDataLeakageColumns(
      df: DataFrame, 
      keepRelativeTime: Boolean = false,
      selectedWeatherColumns: Option[Seq[String]] = None
    ): DataFrame = {
      
      val leakagePatterns = Set(
        "DEP_TIME",
        "DEP_DELAY",
        "DEP_DELAY_NEW",
        "ARR_TIME",
        "ARR_DELAY",
        "ARR_DELAY_NEW",
        "CARRIER_DELAY",
        "WEATHER_DELAY",
        "NAS_DELAY",
        "SECURITY_DELAY",
        "LATE_AIRCRAFT_DELAY",
        "ACTUAL_ELAPSED_TIME",
        "AIR_TIME",
        "CANCELLED",
        "CANCELLATION_CODE",
        "DIVERTED",
        "DISTANCE",
        "feature_departure_hour_rounded",
        "feature_utc_departure_hour_rounded",
        "feature_arrival_hour_rounded",
        "feature_utc_arrival_hour_rounded",
        "feature_arrival_hour",
        "feature_utc_arrival_hour",
        "feature_departure_hour",
        "feature_utc_departure_hour",
        "UTC_ARR_TIME",
        "UTC_ARR_DATE"
      )
      
      val currentColumns = df.columns.toSet
      val columnsToRemove = currentColumns.intersect(leakagePatterns)
      val columnsToKeep = currentColumns.diff(columnsToRemove)
      
      if (columnsToRemove.nonEmpty) {
        println(s"[Anti-Leakage] Colonnes supprimées : ${columnsToRemove.toSeq.sorted.mkString(", ")}")
      }
      
      val dfCleaned = df.select(columnsToKeep.toSeq.sorted.map(col): _*)
      
      // Déterminer les colonnes météo à garder (sans WBAN, Date, Time qui sont obligatoires)
      val weatherColsToKeep = selectedWeatherColumns match {
        case Some(cols) => 
          // Filtrer pour ne garder que les colonnes demandées (sans les colonnes techniques)
          cols.filterNot(c => c == "WBAN" || c == "Date" || c == "Time")
        case None =>
          // Si None, on doit inférer les colonnes à partir du premier élément du premier array
          // Pour l'instant, on va juste supprimer Time et Date, et garder le reste
          Seq() // On ne peut pas le savoir sans inspecter le DataFrame
      }
      
      // Nettoyer les observations météo
      val weatherObsCols = dfCleaned.columns.filter(_.endsWith("_weather_observations"))
      
      var result = dfCleaned
      
      if (weatherObsCols.nonEmpty) {
        // Construire dynamiquement le struct en fonction des colonnes sélectionnées
        val structFields = if (weatherColsToKeep.nonEmpty) {
          // Utiliser les colonnes explicitement demandées
          weatherColsToKeep.map(colName => s"obs.$colName as $colName").mkString(", ")
        } else {
          // Mode fallback : garder toutes les colonnes sauf WBAN, Date, Time
          // On va devoir inspecter le schema du premier array
          val sampleRow = dfCleaned.select(weatherObsCols.head).first()
          val weatherArray = sampleRow.getAs[Seq[_]](0)
          
          if (weatherArray.nonEmpty) {
            val firstObs = weatherArray.head.asInstanceOf[org.apache.spark.sql.Row]
            val schema = firstObs.schema
            
            schema.fields
              .map(_.name)
              .filterNot(name => name == "WBAN" || name == "Date" || name == "Time")
              .map(colName => s"obs.$colName as $colName")
              .mkString(", ")
          } else {
            // Array vide, on ne peut rien faire
            ""
          }
        }
        
        if (keepRelativeTime && structFields.nonEmpty) {
          // Garder Time mais le transformer en index relatif
          weatherObsCols.foreach { weatherCol =>
            result = result.withColumn(
              weatherCol,
              expr(s"""
                transform(
                  $weatherCol,
                  obs -> struct(
                    $structFields,
                    cast((obs.Time - element_at($weatherCol, -1).Time) / 100 as int) as hours_before_flight
                  )
                )
              """)
            )
          }
          println(s"[Anti-Leakage] Time transformé en hours_before_flight (index relatif)")
          println(s"[Anti-Leakage] Champs conservés dans observations météo : ${weatherColsToKeep.mkString(", ")}, hours_before_flight")
          
        } else if (structFields.nonEmpty) {
          // Supprimer complètement Time et Date
          weatherObsCols.foreach { weatherCol =>
            result = result.withColumn(
              weatherCol,
              expr(s"""
                transform(
                  $weatherCol,
                  obs -> struct($structFields)
                )
              """)
            )
          }
          println(s"[Anti-Leakage] Champs supprimés des observations météo : WBAN, Time, Date")
          println(s"[Anti-Leakage] Champs conservés dans observations météo : ${weatherColsToKeep.mkString(", ")}")
        }
      }
      
      result
    }
  /**
   * Jointure spatio-temporelle pour UN aéroport (origine OU destination)
   */
  def joinWeatherForAirport(
    flightDF: DataFrame,
    weatherDF: DataFrame,
    airportType: String,
    weatherDepthHours: Int,
    selectedFlightColumns: Option[Seq[String]] = None,
    selectedWeatherColumns: Option[Seq[String]] = None
  ): DataFrame = {
    
    require(
      airportType == "origin" || airportType == "destination",
      "airportType doit être 'origin' ou 'destination'"
    )
    
    require(weatherDepthHours > 0 && weatherDepthHours <= 24, 
      s"weatherDepthHours doit être entre 1 et 24, valeur fournie: $weatherDepthHours")
    
    import flightDF.sparkSession.implicits._
    
    val isOrigin = airportType == "origin"
    
    // Déterminer les colonnes à utiliser selon le type d'aéroport
    val (wbanCol, dateCol, timeRoundedCol) = if (isOrigin) {
      ("ORIGIN_WBAN", "UTC_FL_DATE", "feature_utc_departure_hour_rounded")
    } else {
      ("DEST_WBAN", "UTC_ARR_DATE", "feature_utc_arrival_hour_rounded")
    }
    
    // Déterminer les colonnes météo à inclure dans le struct
    val weatherColsToKeep = selectedWeatherColumns match {
      case Some(cols) => 
        val mandatory = Seq("WBAN", "Date", "Time")
        (mandatory ++ cols).distinct
      case None => 
        weatherDF.columns.toSeq
    }
    
    // ============================================
    // PHASE MAP - Préparation des données
    // ============================================
    
    val weatherTagged = weatherDF
      .withColumn("Time", col("Time").cast("int"))
      .withColumn("table_tag", lit("OT"))
      .withColumn("join_key_wban", col("WBAN"))
      .withColumn("join_key_date", col("Date"))
    
    val flightTagged = flightDF
      .filter(col(wbanCol).isNotNull && col(timeRoundedCol).isNotNull)
      .withColumn("table_tag", lit("FT"))
      .withColumn("join_key_wban", col(wbanCol))
      .withColumn("join_key_date", col(dateCol))
      .withColumn("time_rounded", col(timeRoundedCol))
      
    val flightsNeedingDuplication = flightTagged
      .filter(col("time_rounded") / 100 < weatherDepthHours)
      .withColumn("join_key_date", date_sub(col("join_key_date"), 1))
    
    val flightComplete = flightTagged.union(flightsNeedingDuplication)
    
    // ============================================
    // PHASE SHUFFLE - Préparation pour jointure
    // ============================================
    
    val weatherForJoin = weatherTagged
      .select(
        Seq(
          col("join_key_wban").as("key_wban"),
          col("join_key_date").as("key_date"),
          col("table_tag")
        ) ++ weatherColsToKeep.map(c => col(c)): _*
      )
    
    val allFlightCols = flightComplete.columns.toSeq
    val excludeCols = Set("table_tag", "join_key_wban", "join_key_date", "time_rounded")
    val flightDataCols = allFlightCols.filterNot(excludeCols.contains)
    
    val flightForJoin = flightComplete
      .select(
        Seq(
          col("join_key_wban").as("key_wban"),
          col("join_key_date").as("key_date"),
          col("time_rounded")
        ) ++ flightDataCols.map(c => col(c)): _*
      )
    
    // ============================================
    // PHASE REDUCE - Jointure et Agrégation
    // ============================================
    
    val weatherStructCols = weatherColsToKeep.map(c => col(c))
    
    val weatherGrouped = weatherForJoin
      .groupBy("key_wban", "key_date")
      .agg(
        collect_list(
          struct(weatherStructCols: _*)
        ).as("all_weather_obs")
      )
    
    val joined = flightForJoin
      .join(
        weatherGrouped,
        flightForJoin("key_wban") === weatherGrouped("key_wban") &&
        flightForJoin("key_date") === weatherGrouped("key_date"),
        "left"
      )
    
    val weatherColName = s"${airportType}_weather_observations"
    
    val joinedWithFiltered = joined
      .withColumn("required_hours",
        expr(s"""
          transform(
            sequence(0, $weatherDepthHours),
            i -> cast((time_rounded / 100 - ($weatherDepthHours - i) + 24) % 24 * 100 as int)
          )
        """)
      )
      .withColumn(weatherColName,
        expr("""
          filter(
            all_weather_obs,
            obs -> array_contains(required_hours, obs.Time)
          )
        """)
      )
    
    // ============================================
    // SÉLECTION FINALE DES COLONNES
    // ============================================
    
    val finalFlightCols = selectedFlightColumns match {
      case Some(cols) =>
        val colsSet = cols.toSet
        
        val requiredCols = Set(
          "ORIGIN_WBAN", "UTC_FL_DATE", "feature_utc_departure_hour_rounded",
          "DEST_WBAN", "UTC_ARR_DATE", "feature_utc_arrival_hour_rounded"
        )
        
        val missingRequired = requiredCols
          .filter(flightDataCols.contains)
          .diff(colsSet)
        
        var finalCols = (cols ++ missingRequired).distinct
        
        if (!isOrigin && 
            flightDataCols.contains("origin_weather_observations") && 
            !colsSet.contains("origin_weather_observations")) {
          finalCols = (finalCols :+ "origin_weather_observations").distinct
        }
        
        if (missingRequired.nonEmpty) {
          println(s"[FlightWeatherJoinner] Colonnes ajoutées automatiquement pour $airportType: ${missingRequired.mkString(", ")}")
        }
        
        finalCols
        
      case None =>
        flightDataCols
    }
    
    val finalColsSeq = (finalFlightCols :+ weatherColName).map(c => col(c))
    
    val result = joinedWithFiltered
      .select(finalColsSeq: _*)
      .filter(size(col(weatherColName)) > 0)
    
    result
  }
}

defined object FlightWeatherJoinner


In [88]:
// Chargement de Flight Data
val processedFlightPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightDataAll = spark.read.parquet(processedFlightPath)
val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDataAll = spark.read.parquet(processedWeatherPath)

val selectedFlightCols = Seq(
  "FL_DATE",
  "OP_CARRIER_AIRLINE_ID",
  "OP_CARRIER_FL_NUM",
  "ORIGIN_AIRPORT_ID",
  "DEST_AIRPORT_ID",
  "ORIGIN_WBAN",
  "DEST_WBAN",
  "label_is_delayed_15min"
)

val selectedWeatherCols = Seq(
  "SkyCondition",
  "Visibility"
)

// Démarrer le chronomètre
val startTime = System.currentTimeMillis()

val joinedData2 = FlightWeatherJoinner.joinFlightsWithWeather(
  flightDataAll, 
  weatherDataAll,
  weatherDepthHours = 12,  
  removeLeakageColumns = true,  
  selectedFlightColumns = Some(selectedFlightCols),
  selectedWeatherColumns = Some(selectedWeatherCols)
)

// IMPORTANT : Forcer l'exécution avec une action
val count = joinedData2.count()

// Arrêter le chronomètre
val endTime = System.currentTimeMillis()
val executionTime = (endTime - startTime) / 1000.0

println(s"=" * 80)
println(s"Temps d'exécution : $executionTime secondes")
println(s"Nombre de lignes : $count")
println(s"=" * 80)

[FlightWeatherJoinner] Colonnes ajoutées automatiquement pour origin: feature_utc_departure_hour_rounded, UTC_ARR_DATE, UTC_FL_DATE, feature_utc_arrival_hour_rounded
[FlightWeatherJoinner] Colonnes ajoutées automatiquement pour destination: feature_utc_departure_hour_rounded, UTC_ARR_DATE, UTC_FL_DATE, feature_utc_arrival_hour_rounded
[Anti-Leakage] Colonnes supprimées : UTC_ARR_DATE, feature_utc_arrival_hour_rounded, feature_utc_departure_hour_rounded
[Anti-Leakage] Time transformé en hours_before_flight (index relatif)
[Anti-Leakage] Champs conservés dans observations météo : SkyCondition, Visibility, hours_before_flight
Temps d'exécution : 4.261 secondes
Nombre de lignes : 204386


processedFlightPath = /home/jovyan/work/output/common/data/processed_flights.parquet
flightDataAll = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]
processedWeatherPath = /home/jovyan/work/output/common/data/processed_weather.parquet
weatherDataAll = [WBAN: string, Date: date ... 65 more fields]
selectedFlightCols = List(FL_DATE, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID, ORIGIN_WBAN, DEST_WBAN, label_is_delayed_15min)
selectedWeatherCols = List(SkyCondition, Visibility)
startTime = 1760250533292
joinedData2 = [DEST_AIRPORT_ID: int, DEST_WBAN: string ... 9 more fields]
count = 204386


endT...


204386